In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle

In [ ]:
!kaggle datasets download -d gwongz/bears-4



 97% 121M/125M [00:04<00:00, 36.2MB/s]
100% 125M/125M [00:04<00:00, 30.7MB/s]


In [ ]:
!unzip /content/bears-4.zip

Archive:  /content/bears-4.zip
  inflating: bears/black/0069232b-9de7-4b03-82fb-ea4890a4c689.jpg  
  inflating: bears/black/058d2e6e-10bf-498f-b3b0-e3bb23a15ac1.jpg  
  inflating: bears/black/059f6854-e659-43cf-b9c0-e50218b09a53.jpg  
  inflating: bears/black/10c6e6e8-699a-484b-8940-6fdba7837e00.jpg  
  inflating: bears/black/10edc5d2-6b82-4431-beec-bc00510463c3.jpg  
  inflating: bears/black/238f0d54-7a63-4e37-96d8-2ee64ef1aced.jpg  
  inflating: bears/black/241b24a7-7fc5-46f1-bb79-63e4090fcfe8.jpg  
  inflating: bears/black/274a1d87-a2a3-4ac2-9a80-6f7fe487f52a.jpg  
  inflating: bears/black/2d06dee0-d7c6-41d9-aeff-1cce795158e5.jpg  
  inflating: bears/black/2f7c67bf-f93b-40ae-8d74-69395d2cd394.jpg  
  inflating: bears/black/345c0d84-8d23-44f2-bda1-b41513ebb9e6.jpeg  
  inflating: bears/black/36baf05c-f506-438e-9945-f4d60e0bcf67.png  
  inflating: bears/black/3905003f-8510-46f6-9757-5655487d128e.jpg  
  inflating: bears/black/3ce7fd7b-37db-4917-aa22-8a55e35b9d78.jpg  
  inflating: bea

In [ ]:
import os
root_dir = '/content/bears'
class_counts = {}

for class_name in os.listdir(root_dir):
    class_dir = os.path.join(root_dir, class_name)
    if os.path.isdir(class_dir):
        num_images = len(os.listdir(class_dir))
        class_counts[class_name] = num_images


for class_name, count in class_counts.items():
    print(f"{class_name}: {count} images")


red_panda: 44 images
polar: 47 images
giant_panda: 47 images
black: 50 images


In [ ]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

#Tain and Test split

In [ ]:
from sklearn.model_selection import train_test_split

images_directory = "/content/bears"
classes = ['black', 'giant_panda', 'polar', 'red_panda']

output_directory = "/content/output"


os.makedirs(output_directory, exist_ok=True)

train_directory = os.path.join(output_directory, "train")
test_directory = os.path.join(output_directory, "test")
os.makedirs(train_directory, exist_ok=True)
os.makedirs(test_directory, exist_ok=True)


test_size = 0.2


for class_name in classes:
    class_folder = os.path.join(images_directory, class_name)

    images = os.listdir(class_folder)

    train_images, test_images = train_test_split(images, test_size=test_size, random_state=42)


    for image_name in train_images:
        source_path = os.path.join(class_folder, image_name)
        destination_path = os.path.join(train_directory, class_name, image_name)
        os.makedirs(os.path.dirname(destination_path), exist_ok=True)
        os.rename(source_path, destination_path)


    for image_name in test_images:
        source_path = os.path.join(class_folder, image_name)
        destination_path = os.path.join(test_directory, class_name, image_name)
        os.makedirs(os.path.dirname(destination_path), exist_ok=True)
        os.rename(source_path, destination_path)

print("Train-test split completed.")

Train-test split completed.


#CNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Define constants
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 20
NUM_CLASSES = 4
EPOCHS = 10

In [ ]:
# Define directory paths
train_dir = '/content/output/train'
validation_dir = '/content/output/test'

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')


Found 149 images belonging to 4 classes.
Found 39 images belonging to 4 classes.


In [ ]:
# Initialize the CNN
model = Sequential()

In [ ]:

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(units=128, activation='relu'))

In [ ]:
model.add(Dense(units=NUM_CLASSES, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE)



Epoch 1/10
7/7 [==============================] - 12s 1s/step - loss: 1.4386 - accuracy: 0.2636 - val_loss: 1.3454 - val_accuracy: 0.4000
Epoch 2/10
7/7 [==============================] - 8s 1s/step - loss: 1.2904 - accuracy: 0.4264 - val_loss: 1.1464 - val_accuracy: 0.6500
Epoch 3/10
7/7 [==============================] - 8s 1s/step - loss: 1.0068 - accuracy: 0.7054 - val_loss: 0.7802 - val_accuracy: 0.7000
Epoch 4/10
7/7 [==============================] - 7s 1s/step - loss: 0.8811 - accuracy: 0.6124 - val_loss: 0.8654 - val_accuracy: 0.6500
Epoch 5/10
7/7 [==============================] - 8s 1s/step - loss: 0.9937 - accuracy: 0.5969 - val_loss: 0.7377 - val_accuracy: 0.8500
Epoch 6/10
7/7 [==============================] - 8s 1s/step - loss: 0.8145 - accuracy: 0.6589 - val_loss: 0.7285 - val_accuracy: 0.6500
Epoch 7/10
7/7 [==============================] - 8s 1s/step - loss: 0.7030 - accuracy: 0.7519 - val_loss: 0.9110 - val_accuracy: 0.6500
Epoch 8/10
7/7 [========================

In [ ]:

test_loss, test_accuracy = model.evaluate(validation_generator)


print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


2/2 [==============================] - 3s 941ms/step - loss: 0.5490 - accuracy: 0.7436
Test Loss: 0.5490350723266602
Test Accuracy: 0.7435897588729858


In [ ]:
# Save the model
model.save("bear_classifier.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#testing data

In [ ]:
class_names = ['black', 'giant_panda', 'polar', 'red_panda']

In [ ]:
#testing the black black
img_path = '/content/output/test/black/058d2e6e-10bf-498f-b3b0-e3bb23a15ac1.jpg'
img = image.load_img(img_path, target_size=(64, 64))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.
predictions = model.predict(img_array)

predicted_class = class_names[np.argmax(predictions)]

print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 17ms/step
Predicted class: black


In [ ]:
#testing the class giant_panda
img_path = '/content/output/test/giant_panda/232794c8-6995-499b-9907-fd8be48be12f.jpg'
img = image.load_img(img_path, target_size=(64, 64))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.
predictions = model.predict(img_array)

predicted_class = class_names[np.argmax(predictions)]

print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 19ms/step
Predicted class: giant_panda


In [ ]:
#testing the class polar
img_path = '/content/output/test/polar/070a8806-3bfb-44d3-bb41-a8239b98b427.jpg'
img = image.load_img(img_path, target_size=(64, 64))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.
predictions = model.predict(img_array)

predicted_class = class_names[np.argmax(predictions)]

print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 28ms/step
Predicted class: polar


In [ ]:
#testing the class red_panda
img_path = '/content/output/train/red_panda/471c9601-8a43-4584-9b49-15f0d5a3d0b4.jpg'
img = image.load_img(img_path, target_size=(64, 64))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.
predictions = model.predict(img_array)

predicted_class = class_names[np.argmax(predictions)]

print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 20ms/step
Predicted class: red_panda


#vgg16

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 20
NUM_CLASSES = 4
EPOCHS = 10

In [ ]:
train_dir = '/content/output/train'
validation_dir = '/content/output/test'

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 149 images belonging to 4 classes.
Found 39 images belonging to 4 classes.


In [ ]:
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))


In [ ]:
for layer in vgg16_model.layers:
    layer.trainable = False

In [ ]:
model_vgg = Sequential()

In [ ]:
model_vgg.add(vgg16_model)

In [ ]:
model_vgg.add(Flatten())
model_vgg.add(Dense(units=128, activation='relu'))
model_vgg.add(Dense(units=NUM_CLASSES, activation='softmax'))


In [ ]:
model_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model_vgg.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE)

Epoch 1/10
7/7 [==============================] - 17s 2s/step - loss: 4.2310 - accuracy: 0.4109 - val_loss: 1.7787 - val_accuracy: 0.3500
Epoch 2/10
7/7 [==============================] - 9s 1s/step - loss: 1.2786 - accuracy: 0.4806 - val_loss: 0.8012 - val_accuracy: 0.7500
Epoch 3/10
7/7 [==============================] - 10s 2s/step - loss: 0.4982 - accuracy: 0.7984 - val_loss: 0.6038 - val_accuracy: 0.6500
Epoch 4/10
7/7 [==============================] - 10s 1s/step - loss: 0.2463 - accuracy: 0.9302 - val_loss: 0.6539 - val_accuracy: 0.8500
Epoch 5/10
7/7 [==============================] - 10s 1s/step - loss: 0.1801 - accuracy: 0.9457 - val_loss: 0.4311 - val_accuracy: 0.7500
Epoch 6/10
7/7 [==============================] - 10s 1s/step - loss: 0.1032 - accuracy: 0.9690 - val_loss: 1.2198 - val_accuracy: 0.6500
Epoch 7/10
7/7 [==============================] - 12s 2s/step - loss: 0.0782 - accuracy: 0.9845 - val_loss: 1.0265 - val_accuracy: 0.6500
Epoch 8/10
7/7 [===================

In [ ]:
# Save the model
model_vgg.save("vgg_bear_classifier.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#testing

In [ ]:
# testing the class
img_path = '/content/output/test/black/058d2e6e-10bf-498f-b3b0-e3bb23a15ac1.jpg'
img = image.load_img(img_path, target_size=(224, 224))  # VGG16 expects input images of size 224x224
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
predictions = model_vgg.predict(x)

predicted_class = class_names[np.argmax(predictions)]

print("Predicted class:", predicted_class)

1/1 [==============================] - 1s 1s/step
Predicted class: red_panda


In [ ]:
# testing the class
img_path = '/content/output/test/giant_panda/232794c8-6995-499b-9907-fd8be48be12f.jpg'
img = image.load_img(img_path, target_size=(224, 224))  # VGG16 expects input images of size 224x224
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
predictions = model_vgg.predict(x)

predicted_class = class_names[np.argmax(predictions)]

print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 20ms/step
Predicted class: red_panda


In [ ]:
# testing the class
img_path = '/content/output/test/polar/888e38ff-5f8d-47e9-9b5d-b1ba02f529ac.jpeg'
img = image.load_img(img_path, target_size=(224, 224))  # VGG16 expects input images of size 224x224
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
predictions = model_vgg.predict(x)

predicted_class = class_names[np.argmax(predictions)]

print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 19ms/step
Predicted class: red_panda


In [ ]:
# testing the class
img_path = '/content/output/train/polar/221f009b-4cb9-4189-9fd1-d511d7202280.jpg'
img = image.load_img(img_path, target_size=(224, 224))  # VGG16 expects input images of size 224x224
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
predictions = model_vgg.predict(x)

predicted_class = class_names[np.argmax(predictions)]

print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 43ms/step
Predicted class: red_panda


#Tuning the Vgg16 model

In [ ]:

IMAGE_SIZE = (64, 64)
BATCH_SIZE = 32
NUM_CLASSES = 4
EPOCHS = 20


train_dir = '/content/output/train'
validation_dir = '/content/output/test'


train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')


Found 149 images belonging to 4 classes.
Found 39 images belonging to 4 classes.


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
custom_vgg_model = Sequential()

In [ ]:
custom_vgg_model.add(base_model)


In [ ]:
custom_vgg_model.add(Flatten())

In [ ]:
custom_vgg_model.add(Dense(units=128, activation='relu'))
custom_vgg_model.add(Dense(units=NUM_CLASSES, activation='softmax'))


In [ ]:
custom_vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = custom_vgg_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE)

Epoch 1/20
4/4 [==============================] - 11s 3s/step - loss: 1.4097 - accuracy: 0.3761 - val_loss: 1.1758 - val_accuracy: 0.4062
Epoch 2/20
4/4 [==============================] - 7s 2s/step - loss: 0.9404 - accuracy: 0.6484 - val_loss: 1.0199 - val_accuracy: 0.4688
Epoch 3/20
4/4 [==============================] - 8s 2s/step - loss: 0.7354 - accuracy: 0.7734 - val_loss: 0.7538 - val_accuracy: 0.7500
Epoch 4/20
4/4 [==============================] - 8s 2s/step - loss: 0.6812 - accuracy: 0.7436 - val_loss: 0.7972 - val_accuracy: 0.6250
Epoch 5/20
4/4 [==============================] - 8s 2s/step - loss: 0.5398 - accuracy: 0.8376 - val_loss: 0.7922 - val_accuracy: 0.6562
Epoch 6/20
4/4 [==============================] - 7s 2s/step - loss: 0.4369 - accuracy: 0.8632 - val_loss: 0.5886 - val_accuracy: 0.7812
Epoch 7/20
4/4 [==============================] - 8s 2s/step - loss: 0.3464 - accuracy: 0.8828 - val_loss: 0.7184 - val_accuracy: 0.6562
Epoch 8/20
4/4 [========================

In [ ]:
img_path = '/content/output/test/black/058d2e6e-10bf-498f-b3b0-e3bb23a15ac1.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.
predictions = custom_vgg_model.predict(x)


class_names = ['black', 'giant_panda', 'polar', 'red_panda']  # Adjust these with your actual class names
predicted_class = class_names[np.argmax(predictions)]


print("Predicted class:", predicted_class)

1/1 [==============================] - 1s 682ms/step
Predicted class: black


In [ ]:
img_path = '/content/output/test/giant_panda/1f35789c-18cb-4047-8333-94112aae7753.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.
predictions = custom_vgg_model.predict(x)


class_names = ['black', 'giant_panda', 'polar', 'red_panda']  # Adjust these with your actual class names
predicted_class = class_names[np.argmax(predictions)]


print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 20ms/step
Predicted class: giant_panda


In [ ]:
img_path = '/content/output/test/polar/49054759-0e77-4957-84ad-ec8ff5998bfb.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.
predictions = custom_vgg_model.predict(x)


class_names = ['black', 'giant_panda', 'polar', 'red_panda']  # Adjust these with your actual class names
predicted_class = class_names[np.argmax(predictions)]


print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 18ms/step
Predicted class: polar


In [ ]:
img_path = '/content/output/test/red_panda/26e401c2-1218-47da-a2c8-9d58312fd6ab.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.
predictions = custom_vgg_model.predict(x)


class_names = ['black', 'giant_panda', 'polar', 'red_panda']  # Adjust these with your actual class names
predicted_class = class_names[np.argmax(predictions)]


print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 19ms/step
Predicted class: red_panda


In [ ]:
# Save the model
custom_vgg_model.save("bear_classifier.h5")

#Inception Model

In [ ]:
# Define constants
IMAGE_SIZE = (299, 299)
BATCH_SIZE = 32
NUM_CLASSES = 4
EPOCHS = 10

In [ ]:
train_dir = '/content/output/train'
validation_dir = '/content/output/test'


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 149 images belonging to 4 classes.
Found 39 images belonging to 4 classes.


In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
custom_inception_model = Sequential()

In [ ]:
custom_inception_model.add(base_model)

In [ ]:
custom_inception_model.add(GlobalAveragePooling2D())


In [ ]:
custom_inception_model.add(Dense(units=128, activation='relu'))
custom_inception_model.add(Dense(units=NUM_CLASSES, activation='softmax'))

In [ ]:
custom_inception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = custom_inception_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE)


Epoch 1/10
4/4 [==============================] - 70s 4s/step - loss: 0.8328 - accuracy: 0.6752 - val_loss: 0.1718 - val_accuracy: 1.0000
Epoch 2/10
4/4 [==============================] - 10s 3s/step - loss: 0.0954 - accuracy: 0.9915 - val_loss: 0.0233 - val_accuracy: 1.0000
Epoch 3/10
4/4 [==============================] - 11s 3s/step - loss: 0.0159 - accuracy: 1.0000 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 4/10
4/4 [==============================] - 13s 3s/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 5/10
4/4 [==============================] - 12s 3s/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0027 - val_accuracy: 1.0000
Epoch 6/10
4/4 [==============================] - 11s 3s/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 10s 3s/step - loss: 4.4672e-04 - accuracy: 1.0000 - val_loss: 6.3235e-04 - val_accuracy: 1.0000
Epoch 8/10
4/4 [==========

In [ ]:
#custom_inception_model.save("My_Model.h5")

In [ ]:
img_path = '/content/output/test/black/058d2e6e-10bf-498f-b3b0-e3bb23a15ac1.jpg'
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.
predictions = custom_inception_model.predict(x)
predicted_class = class_names[np.argmax(predictions)]
print("Predicted class:", predicted_class)

1/1 [==============================] - 3s 3s/step
Predicted class: black


In [ ]:
img_path = '/content/output/test/giant_panda/1f35789c-18cb-4047-8333-94112aae7753.jpg'
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.
predictions = custom_inception_model.predict(x)
predicted_class = class_names[np.argmax(predictions)]
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 44ms/step
Predicted class: giant_panda


In [ ]:
img_path = '/content/output/test/polar/4aee8307-1980-4939-92ed-47f16a4f6ee4.jpg'
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.
predictions = custom_inception_model.predict(x)
predicted_class = class_names[np.argmax(predictions)]
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 25ms/step
Predicted class: polar


In [ ]:
img_path = '/content/output/test/red_panda/26e401c2-1218-47da-a2c8-9d58312fd6ab.jpg'
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.
predictions = custom_inception_model.predict(x)
predicted_class = class_names[np.argmax(predictions)]
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 64ms/step
Predicted class: red_panda
